<a href="https://colab.research.google.com/github/wooohun/BERT-Summarizer/blob/main/BART_Abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies
!pip install transformers==4.28.0
!pip install torch
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14

In [2]:
import pandas as pd
import torch
import nltk
import evaluate
from datasets import load_dataset, load_metric, DatasetDict
from evaluate import evaluator

In [3]:
# install kaggle
!pip install -q kaggle

# get kaggle api token from account -> API -> create new API Token
# move kaggle api token to kaggle folder

!cp -v /content/kaggle.json ~/.kaggle/


cp: cannot stat '/content/kaggle.json': No such file or directory


In [6]:
# Move the Kaggle API key file to the appropriate directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set appropriate permissions to the Kaggle API key file
!chmod 600 ~/.kaggle/kaggle.json
# Download the dataset using Kaggle API
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

# Unzip the downloaded dataset
!unzip newspaper-text-summarization-cnn-dailymail.zip


100% 502M/503M [00:05<00:00, 104MB/s]
100% 503M/503M [00:05<00:00, 94.6MB/s]
Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [7]:
# grab datasets
train = load_dataset("cnn_dailymail",'3.0.0', split="train[:34%]")
test = load_dataset("cnn_dailymail",'3.0.0', split="test[:51%]")
dataset = DatasetDict()
dataset['train'] = train
dataset['test'] = test
# formatted as:
# DatasetDict{
#   train: Dataset{ - len = 287113 * x%
#     features:[]
#     num_rows: int 
#   }
#   test: {} - len = 11490
#
# }



Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/cnn_dailymail-0ae51db9f6a95f14/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [14]:
from transformers import BartTokenizerFast, BartForConditionalGeneration

# using fast tokenizer we initialize our tokenizer object
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
model  = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [15]:
def preprocessing(dataset):
  #article inputs aka full article
  inputs = [article for article in dataset['article']]

  #article target aka summarization
  targets = [target for target in dataset['highlights']]

  #although tokenizing targets is optional, it helps with consistency among the tokenization process
  tokenized_inputs = tokenizer(inputs, text_target=targets, max_length = 1024, truncation=True)

  # # set tokenizer to encode labels
  # with tokenizer.as_target_tokenizer():
  #   labels = tokenizer(dataset['highlights'], max_length = 128, truncation=True)
  return tokenized_inputs

In [19]:
# process all datasets in batches of 16 input & target = 32 articles, using fast tokenizer for efficiency
processed_dataset = dataset.map(preprocessing, batched = True, batch_size = 16)

# Benefits of batch processing for tokenization
# batch processing for tokenization can help reduce overfitting so it generalizes better of a subset of data
# speeds training
# smaller batch size has faster convergence than larger batch size since parameters are updated frequently

Map:   0%|          | 0/97618 [00:00<?, ? examples/s]

Map:   0%|          | 0/5860 [00:00<?, ? examples/s]

In [20]:
# we decode the first target article summary and see the summarization 
tokenizer.batch_decode(processed_dataset['train']['labels'][0], skip_special_tokens=True)

['',
 'B',
 'ishop',
 ' John',
 ' Fold',
 'a',
 ',',
 ' of',
 ' North',
 ' Dakota',
 ',',
 ' is',
 ' taking',
 ' time',
 ' off',
 ' after',
 ' being',
 ' diagnosed',
 '.',
 '\n',
 'He',
 ' contracted',
 ' the',
 ' infection',
 ' through',
 ' contaminated',
 ' food',
 ' in',
 ' Italy',
 '.',
 '\n',
 'Church',
 ' members',
 ' in',
 ' Fargo',
 ',',
 ' Grand',
 ' For',
 'ks',
 ' and',
 ' Jam',
 'est',
 'own',
 ' could',
 ' have',
 ' been',
 ' exposed',
 '.',
 '']

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

In [ ]:
!pip install --upgrade accelerate

In [ ]:
import optuna
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# Create the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Create Optuna study and optimize
study = optuna.create_study(direction='maximize')

# Define the objective function for Optuna
def objective(trial):
    # our options of hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-4)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-4)
    batch_size = trial.suggest_categorical('batch_size', [2, 4, 8])
    logging_steps = trial.suggest_int('logging_steps', 100, 1000, step=100)

    # Create training arguments
    args = Seq2SeqTrainingArguments(
        output_dir="facebook-bart-base-finetuned-cnn-dailymail",
        learning_rate=learning_rate,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        save_total_limit=3,
        predict_with_generate=True,
        fp16=True,
        logging_steps=logging_steps
    )
    # Update the hyperparameters within the objective function
    args.learning_rate = best_params['learning_rate']
    args.weight_decay = best_params['weight_decay']
    args.per_device_train_batch_size = best_params['batch_size']
    args.per_device_eval_batch_size = best_params['batch_size']

    # Create trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=processed_dataset['train'],
        eval_dataset=processed_dataset['test'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=metric_compute
    )

    # Train the model
    trainer.train()

    # Evaluate the trained model and compute performance metric
    eval_result = trainer.evaluate()

    # Calculate the performance metric to optimize (e.g., Rouge)
    performance_metric = eval_result['rouge2']

    # Return the performance metric to Optuna for optimization
    return performance_metric

In [ ]:
# install metrics
!pip install bert_score
!pip install rouge_score

In [ ]:
# load metrics
rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')

In [ ]:
import numpy as np

# metric computation function to pass into trainer object
def metric_compute(predicted):
    predictions, labels = predicted
    # decode predictions, labels for metric computation
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # batch_decode returns list of tokens, use nltk to convert list of tokens to list of sentences
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # compute metrics (e.g., Rouge)
    rouge_res = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in rouge_res.items()}

    return {key: round(val, 4) for key, val in result.items()}

# Start the optimization process
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params

In [ ]:
!pip install pynvml nvidia-ml-py3

In [ ]:
# we can see the computational efficiency of the program
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()